In [ ]:
# hide
# default_exp core
# default_cls_lvl 3
# from nbdev.showdoc import show_doc

# sniptool.core

> magic commands for snippets management, supported by a sqlite db.

In [ ]:
# exporti
import os
import datetime
import pandas as pd
from glob import glob
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
def getmtime_ms(path):
    mtime = os.path.getmtime(path)
    stamp = datetime.datetime.fromtimestamp(mtime, tz=datetime.timezone.utc)
    return int(round(stamp.timestamp()))

def get_change_file(regex, last, this):
    if not os.path.isfile(last):
        os.system(f'touch {last}')
    nb_list = glob(regex)
#     print('nb_list', nb_list, regex)
    mtime = list(map(getmtime_ms, nb_list))
    df = (pd.DataFrame(zip(nb_list, mtime), columns=["path", "mtime"])
          .sort_values(by='path')
          .to_csv(this, sep='|', index=False, header=False))
#     print(f'diff -y --suppress-common-lines {last} {this}')
    diff_ret = os.popen(f'diff -y --suppress-common-lines {last} {this}').read()
    for line in diff_ret.splitlines():
        file = line.split('|')[0].split()[-1]
        yield file
    os.rename(this, last)

In [ ]:
import base64
import os
import sys
import json
import pandas as pd
import yaml
import logging
import pycodestyle as pycodestyle_module
import tempfile
from glob import glob
from PIL import Image
from io import BytesIO, StringIO
from os.path import expanduser
from pathlib import Path
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
from sqlalchemy import create_engine, Column, String, Float, Text
from sqlalchemy.dialects.sqlite import insert, dialect
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [19]:
# hide
# define tables
class NBfiles(Base):
    __tablename__ = "nbfiles"
    path = Column(String(256), primary_key=True)
    mtime = Column(Float(64))


class Snippets(Base):
    __tablename__ = "snippets"
    tags = Column(String(256))
    desc = Column(String(256), primary_key=True)
    source = Column(Text)
    url = Column(String(256))


def _snippet_parser(path):
    sys.stdout.write(f"updating snippets from {path}\n")
    sys.stdout.flush()
    notebook = json.load(open(path))

    for cell in notebook["cells"]:
        if not cell["source"]:
            continue
        head = cell["source"][0]
        if head.startswith("# @"):
            desc, *tags = tuple(head.lstrip("# @").split("#"))
            tags = ";".join(tags) if tags else ""
            source = "".join(cell["source"][1:])
            url = ""
            yield (
                dict(
                    tags=tags,
                    desc=desc,
                    source=source,
                    url=url,
                )
            )


def _get_file_list(targets, exclude=["__pycache__", ".ipynb_checkpoints"]):
    """ "
    This function recursively prints all contents of a pathlib.Path object
    """
    import configparser

    nb_list = [(None, None)]
    for target in targets:
        target = Path(target)
        cfg1, cfg2 = target / "ghconfig", target / ".git/config"
        if cfg1.is_file():  # has ghconfig
            cfg = yaml.load(open(str(cfg1)), Loader=yaml.BaseLoader)
            root = cfg["repo"].rstrip(".git") + "/" + target.name
        elif cfg2.is_file():
            config = configparser.ConfigParser()
            config.read(str(cfg2))
            root = config['remote "origin"']["url"].rstrip(".git") + "/" + target.name
        else:
            root = None
        for e in exclude:
            if target.name.find(e) != -1:
                return list(map(list, zip(*nb_list)))
        for file in target.iterdir():
            if file.is_dir():
                if "." in file.name:
                    continue
                nb_list += _get_file_list([file.absolute().as_posix()], exclude=exclude)
            elif file.name.endswith(".ipynb"):
                nb_list.append((file.absolute().as_posix(), root))
    # remove invalid lines
    nb_list = list(filter(lambda x: x[1], nb_list))
    return list(map(list, zip(*nb_list)))


# _get_file_list(['/home/jovyan/helper'], exclude=['.ipynb_checkpoints'])

# def _compile_query(query):
#     compiler = (
#         query.compile if not hasattr(
#             query, "statement") else query.statement.compile
#     )
#     return compiler(dialect=dialect())


def _get_snippets_file_list(dirs):
    include = [d for d in dirs if not d.startswith("!")]
    exclude = [d[1:] for d in dirs if d.startswith("!")]
    nb_list, links = _get_file_list(include, exclude)
    mtime = list(map(os.path.getmtime, nb_list))
    return pd.DataFrame(zip(nb_list, mtime, links), columns=["path", "mtime", "url"])


def _disp(snippets, short=False):
    display(
        HTML(
            """
    <style>
    {pygments_css}
    </style>
    """.format(
                pygments_css=HtmlFormatter().get_style_defs(".highlight")
            )
        )
    )

    for _, row in snippets.iterrows():
        if short:
            content = row["desc"]
        else:
            content = row["desc"] + "\n" + row["source"][:80]
        display(
            HTML(data=highlight(f"({_})" + content, PythonLexer(), HtmlFormatter()))
        )


def _upsert(df, conn=None, model=None):
    table = model.__table__
    stmt = insert(table).values(df.to_dict(orient="records"))
    update_cols = [c.name for c in table.c if c not in list(table.primary_key.columns)]
    on_conflict_stmt = stmt.on_conflict_do_update(
        index_elements=table.primary_key.columns,
        set_={c.key: c for c in stmt.excluded if c.key in update_cols},
    )
    conn.execute(on_conflict_stmt)

In [ ]:
@magics_class
class SnippetsMagics(Magics):
    """snip magic for snippets management.
    Provides the %snip magic.
    """

    def __init__(self, shell):
        super(SnippetsMagics, self).__init__(shell=shell)
        dotfile = "~/config.yaml"
        cfg = yaml.load(open(expanduser(dotfile)), Loader=yaml.BaseLoader)
        self.db = cfg["config_dir"] + "/SnippetDB.db"
        self.search_dirs = cfg["search_dirs"]
        self.engine = create_engine(f"sqlite:///{self.db}")
        Base.metadata.create_all(self.engine)
        self.snippets = pd.read_sql("snippets", self.engine)

    def update(self):
        nb_to_search = _get_snippets_file_list(self.search_dirs)
        nb_exists = pd.read_sql("nbfiles", self.engine).rename(
            columns={"mtime": "utime"}
        )
        #         print(nb_to_search, nb_exists, nb_exists.empty)
        if nb_exists.empty:
            nb_to_update = nb_to_search.assign(isnew=True)
        else:
            nb_to_update = (
                nb_exists.merge(nb_to_search, how="right")
                .fillna(0)
                .assign(isnew=lambda x: x.mtime > x.utime + 1)
                .query("isnew == True")
            )
        if nb_to_update.empty:
            print('nothing update.')
            return
        nestedList = nb_to_update.path.apply(_snippet_parser).to_list()
        (
            pd.DataFrame([item for sublist in nestedList for item in sublist])
            .pipe(
                _upsert, conn=self.engine, model=Snippets
            )
        )
        if not nb_to_update.empty:
            (
                nb_to_update[["path", "mtime"]].pipe(
                    _upsert, conn=self.engine, model=NBfiles
                )
            )
        self.snippets = pd.read_sql("snippets", self.engine)

    def reset(self):
        self.engine.dispose()
        os.remove(self.db)
        self.engine = create_engine(f"sqlite:///{self.db}")
        Base.metadata.create_all(self.engine)
        self.update()

    def info(self):
        snippets = pd.read_sql("snippets", self.engine)
        nb_exists = pd.read_sql("nbfiles", self.engine)
        print(
            "{} snippets, {} notebooks in SnippetDB".format(
                snippets.shape[0], nb_exists.shape[0]
            )
        )

    @line_magic
    def snip(self, parameter_s=""):
        """snippets management

        Usage:
        * ``%snip``          - total numbers of all snippets and notebooks
        * ``%snip bar``      - search in the db
        * ``%snip -r``       - reset indexing
        * ``%snip -u``       - update db
        * ``%snip <key >-s``       - short output

        New search directories can be inserted into config.yaml['search_dirs']
        """

        opts, argsl = self.parse_options(parameter_s, "rust", mode="string")
        args = argsl.split()
        if "r" in opts:
            self.reset()
        elif "t" in opts:
            print(self.snippets)
        elif "u" in opts:
            self.update()
        elif not args:
            self.info()
        elif args[-1].isdigit():  # load snippets
            pos, q = int(args[-1]), " ".join(args[:-1])
            (
                self.snippets.query(f"desc.str.contains('{q}')", engine="python")
                .pipe(lambda _df: _df.iloc[pos]["source"])
                .pipe(self.shell.set_next_input, replace=False)
            )
        else:
            q = " ".join(args)
            (
                self.snippets.query(f"desc.str.contains('{q}')", engine="python")
                .reset_index()
                .pipe(_disp, short="s" in opts)
            )

In [ ]:
# exporti
@magics_class
class MiscMagics(Magics):
    """snip magic for snippets management.
    Provides the %snip magic.
    """
    @line_magic
    def d(self, parameter_s="", n_cols=5, regex="", cap=False):
        import inspect

        def chunks(names: list, n: int):
            """Yield successive n-sized chunks from lst."""
            r = int(len(names) / n + 0.5)
            for i in range(0, len(names), r):
                yield names[i : i + r]

        def output(items, n_cols=5):
            newcolumns = list(chunks(items, n_cols))
            n_rows, n_columns = len(newcolumns[0]), len(newcolumns)
            newcolumns[-1].extend([""] * (n_rows * n_columns - len(items)))
            array_outputs = [[row[i] for row in newcolumns] for i in range(n_rows)]
            col_width = (
                max(len(word) for row in array_outputs for word in row) + 2
            )  # padding
            for row in array_outputs:
                print("".join(word.ljust(col_width) for word in row))

        opts, argsl = self.parse_options(parameter_s, "cn:s:", mode="string")
        args = argsl.split()
        if "n" in opts:
            n_cols = int(opts["n"])
        if "c" in opts:
            cap = True
        if len(args) > 1:
            regex = args[1]
        for _, f in enumerate(inspect.stack()[4:]):
            if args[0] in f[0].f_locals:
                target = f[0].f_locals[args[0]]
                items = list(filter(lambda x: x[0] != "_", dir(target)))
                if regex:
                    items = list(filter(lambda x: x.find(regex) != -1, items))
                if not cap:
                    items = list(filter(lambda x: x[0].islower(), items))
                output(items, n_cols=n_cols)
                break

    @line_magic
    def lint(self, parameter_s=""):
        opts, argsl = self.parse_options(parameter_s, "abf", mode="string")
        args = argsl.split()
        assert os.path.isfile(args[0])
        if "f" in opts:
            cmd = f"autopep8 --in-place {args[0]}"
            print("autopep8: finished")
        elif "b" in opts:
            cmd = f"nbblack {args[0]}"
        elif "a" in opts:
            cmd = f"pylint --disable=R,C --rcfile ~/.pylintrc {args[0]}"
        else:
            cmd = f"pycodestyle --show-source --ignore E501 {args[0]}"
        print(cmd)
        os.system(cmd)

    @cell_magic
    def runingb(self, line, codes):
        from multiprocessing import Process
        from tempfile import NamedTemporaryFile

        def wrapper(pyfile):
            import subprocess
            import time

            start = time.time()
            proc = subprocess.Popen(["python", pyfile], stdout=subprocess.PIPE)
            print(proc.stdout.readline().decode())
            print("Wall time:", time.time() - start)

        f = NamedTemporaryFile(mode="w", suffix=".py", delete=False)
        #         print(f.name)
        with open(f.name, "w") as fout:
            fout.write(codes)
        p = Process(target=wrapper, args=(f.name,), name="wrapper")
        p.start()

#     @cell_magic
#     def toc(self, ipynb, markdown_content):
#         def mkreader(md):
#             for line in ("\n" + md).split("\n#"):
#                 if line:
#                     yield ("#" + line)

#         cells = json.load(open(ipynb))
#         for source in reversed(list(mkreader(markdown_content))):
#             cell = {
#                 "cell_type": "markdown",
#                 "id": "",
#                 "metadata": {},
#                 "source": [line + "\n" for line in source.splitlines()],
#             }
#             cells["cells"].insert(0, cell)
#         with open(ipynb, "w") as fout:
#             json.dump(cells, fout, indent=2)

#     @line_magic
#     def tidy(self, ipynb: str):
#         cells = json.load(open(ipynb))
#         cnt = 1
#         clean_version = []
#         for cell in cells["cells"]:
#             if cell["cell_type"] == "code":
#                 if not cell["source"]:
#                     continue
#                 if not cell["source"][0].startswith("#"):
#                     continue
#                 cell["execution_count"] = None
#             elif cell["cell_type"] == "markdown":
#                 if not cell["source"]:
#                     continue
#                 headline = cell["source"][0]
#                 if headline.startswith("#") or headline.startswith("<a id="):
#                     if cnt == 1:
#                         print("# TOC")
#                     header = headline.lstrip("# ").strip()
#                     print(f"* [{header}](#cell{cnt})")
#                     cell["source"].insert(1, f'<a id="cell{cnt}"></a>\n')
#                     cnt += 1
#             clean_version.append(cell)
#         out_file = open(ipynb, "w")
#         cells["cells"] = clean_version
#         json.dump(cells, out_file, indent=2)
#         out_file.close()

    @cell_magic
    def pep8(self, line, cell, auto=False):
        """pycodestyle cell magic for pep8"""
        if cell.startswith(("!", "%%", "%")):
            return
        with tempfile.NamedTemporaryFile(mode="r+", delete=False) as f:
            f.write("# The %%pycodestyle cell magic was here\n" + cell)
            f.flush()
            f.close()
        format = "%(row)d:%(col)d: %(code)s %(text)s"
        pycodestyle = pycodestyle_module.StyleGuide(format=format)
        pcs_result = pycodestyle.check_files(paths=[f.name])
        try:
            os.remove(f.name)
        except OSError as e:  # if failed, report it back to the user ##
            logger.error("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# @check package version #rt
!pip list | egrep 'pandas|matplotlib|numpy|seaborn|xarray|rasterio' | sed -e 's/ \+/==/g'

In [1]:
%load_ext sniptool